# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f539c217940>
├── 'a' --> tensor([[ 1.0190,  1.7887, -0.7387],
│                   [-1.1590,  1.3558,  1.3615]])
└── 'x' --> <FastTreeValue 0x7f539c2172b0>
    └── 'c' --> tensor([[-0.9629, -1.7994,  0.7254, -0.2159],
                        [-1.4610, -0.8969, -1.3834, -0.4852],
                        [ 0.3814,  0.0319, -0.3501, -1.5002]])

In [4]:
t.a

tensor([[ 1.0190,  1.7887, -0.7387],
        [-1.1590,  1.3558,  1.3615]])

In [5]:
%timeit t.a

70.8 ns ± 0.089 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f539c217940>
├── 'a' --> tensor([[-0.3739,  0.7221,  0.3413],
│                   [ 0.1029, -1.1335,  0.1932]])
└── 'x' --> <FastTreeValue 0x7f539c2172b0>
    └── 'c' --> tensor([[-0.9629, -1.7994,  0.7254, -0.2159],
                        [-1.4610, -0.8969, -1.3834, -0.4852],
                        [ 0.3814,  0.0319, -0.3501, -1.5002]])

In [7]:
%timeit t.a = new_value

64.1 ns ± 0.0289 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.0190,  1.7887, -0.7387],
               [-1.1590,  1.3558,  1.3615]]),
    x: Batch(
           c: tensor([[-0.9629, -1.7994,  0.7254, -0.2159],
                      [-1.4610, -0.8969, -1.3834, -0.4852],
                      [ 0.3814,  0.0319, -0.3501, -1.5002]]),
       ),
)

In [10]:
b.a

tensor([[ 1.0190,  1.7887, -0.7387],
        [-1.1590,  1.3558,  1.3615]])

In [11]:
%timeit b.a

58.4 ns ± 0.0139 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.0629,  1.9014,  2.8080],
               [-0.6396, -0.3232, -1.9163]]),
    x: Batch(
           c: tensor([[-0.9629, -1.7994,  0.7254, -0.2159],
                      [-1.4610, -0.8969, -1.3834, -0.4852],
                      [ 0.3814,  0.0319, -0.3501, -1.5002]]),
       ),
)

In [13]:
%timeit b.a = new_value

486 ns ± 0.139 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

838 ns ± 2.16 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.7 µs ± 29.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

141 µs ± 434 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

143 µs ± 527 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f52c4b1e7f0>
├── 'a' --> tensor([[[ 1.0190,  1.7887, -0.7387],
│                    [-1.1590,  1.3558,  1.3615]],
│           
│                   [[ 1.0190,  1.7887, -0.7387],
│                    [-1.1590,  1.3558,  1.3615]],
│           
│                   [[ 1.0190,  1.7887, -0.7387],
│                    [-1.1590,  1.3558,  1.3615]],
│           
│                   [[ 1.0190,  1.7887, -0.7387],
│                    [-1.1590,  1.3558,  1.3615]],
│           
│                   [[ 1.0190,  1.7887, -0.7387],
│                    [-1.1590,  1.3558,  1.3615]],
│           
│                   [[ 1.0190,  1.7887, -0.7387],
│                    [-1.1590,  1.3558,  1.3615]],
│           
│                   [[ 1.0190,  1.7887, -0.7387],
│                    [-1.1590,  1.3558,  1.3615]],
│           
│                   [[ 1.0190,  1.7887, -0.7387],
│                    [-1.1590,  1.3558,  1.3615]]])
└── 'x' --> <FastTreeValue 0x7f52c4b1e790>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33 µs ± 64.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f539f2976a0>
├── 'a' --> tensor([[ 1.0190,  1.7887, -0.7387],
│                   [-1.1590,  1.3558,  1.3615],
│                   [ 1.0190,  1.7887, -0.7387],
│                   [-1.1590,  1.3558,  1.3615],
│                   [ 1.0190,  1.7887, -0.7387],
│                   [-1.1590,  1.3558,  1.3615],
│                   [ 1.0190,  1.7887, -0.7387],
│                   [-1.1590,  1.3558,  1.3615],
│                   [ 1.0190,  1.7887, -0.7387],
│                   [-1.1590,  1.3558,  1.3615],
│                   [ 1.0190,  1.7887, -0.7387],
│                   [-1.1590,  1.3558,  1.3615],
│                   [ 1.0190,  1.7887, -0.7387],
│                   [-1.1590,  1.3558,  1.3615],
│                   [ 1.0190,  1.7887, -0.7387],
│                   [-1.1590,  1.3558,  1.3615]])
└── 'x' --> <FastTreeValue 0x7f539f312eb0>
    └── 'c' --> tensor([[-0.9629, -1.7994,  0.7254, -0.2159],
                        [-1.4610, -0.8969, -1.3834, -0.4852],
                 

In [23]:
%timeit t_cat(trees)

30.7 µs ± 49.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

69.9 µs ± 355 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.0190,  1.7887, -0.7387],
                [-1.1590,  1.3558,  1.3615]],
       
               [[ 1.0190,  1.7887, -0.7387],
                [-1.1590,  1.3558,  1.3615]],
       
               [[ 1.0190,  1.7887, -0.7387],
                [-1.1590,  1.3558,  1.3615]],
       
               [[ 1.0190,  1.7887, -0.7387],
                [-1.1590,  1.3558,  1.3615]],
       
               [[ 1.0190,  1.7887, -0.7387],
                [-1.1590,  1.3558,  1.3615]],
       
               [[ 1.0190,  1.7887, -0.7387],
                [-1.1590,  1.3558,  1.3615]],
       
               [[ 1.0190,  1.7887, -0.7387],
                [-1.1590,  1.3558,  1.3615]],
       
               [[ 1.0190,  1.7887, -0.7387],
                [-1.1590,  1.3558,  1.3615]]]),
    x: Batch(
           c: tensor([[[-0.9629, -1.7994,  0.7254, -0.2159],
                       [-1.4610, -0.8969, -1.3834, -0.4852],
                       [ 0.3814,  0.0319, -0.3501, -1.5002]],
         

In [26]:
%timeit Batch.stack(batches)

80.9 µs ± 426 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.0190,  1.7887, -0.7387],
               [-1.1590,  1.3558,  1.3615],
               [ 1.0190,  1.7887, -0.7387],
               [-1.1590,  1.3558,  1.3615],
               [ 1.0190,  1.7887, -0.7387],
               [-1.1590,  1.3558,  1.3615],
               [ 1.0190,  1.7887, -0.7387],
               [-1.1590,  1.3558,  1.3615],
               [ 1.0190,  1.7887, -0.7387],
               [-1.1590,  1.3558,  1.3615],
               [ 1.0190,  1.7887, -0.7387],
               [-1.1590,  1.3558,  1.3615],
               [ 1.0190,  1.7887, -0.7387],
               [-1.1590,  1.3558,  1.3615],
               [ 1.0190,  1.7887, -0.7387],
               [-1.1590,  1.3558,  1.3615]]),
    x: Batch(
           c: tensor([[-0.9629, -1.7994,  0.7254, -0.2159],
                      [-1.4610, -0.8969, -1.3834, -0.4852],
                      [ 0.3814,  0.0319, -0.3501, -1.5002],
                      [-0.9629, -1.7994,  0.7254, -0.2159],
                      [-1.4610, -

In [28]:
%timeit Batch.cat(batches)

150 µs ± 275 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

328 µs ± 5.08 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
